In [ ]:
!pip install astroNN

In [28]:
#importing required libraries
import numpy as np
import keras

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from tensorflow.keras.optimizers import SGD
from astroNN.datasets import galaxy10
from astroNN.datasets.galaxy10 import galaxy10cls_lookup
from sklearn.metrics import classification_report ,confusion_matrix
from tensorflow.keras import utils

In [3]:
def preprocess(images):
    """ Normalize the images
    Parameter
    ---------
    images : list of images
        The images to normalize
    Return
    -------
        Normalized images
    """
    images = images/255

    return images

In [4]:
#loading the dataset
images, labels = galaxy10.load_data()

Galaxy10.h5:  99%|█████████▉| 209M/210M [00:02<00:00, 77.2MB/s]

Downloaded Galaxy10 successfully to /root/.astroNN/datasets/Galaxy10.h5


Galaxy10.h5: 210MB [00:03, 64.9MB/s]                           


In [5]:
# To convert the labels to categorical 10 classes
labels = utils.to_categorical(labels, 10)

#converting the arrays to float type
labels = labels.astype(np.float32)
images = images.astype(np.float32)

In [6]:
#normalizing images
images = preprocess(images)

In [7]:
X = images
y = labels

In [8]:
#splitting the data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [30]:
#building a seq2seq model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3),activation='relu', input_shape=(69,69,3)))
model.add(Conv2D(filters=32,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=256,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(filters=256,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(filters=256,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer = 'he_uniform'))
model.add(Dense(128, activation='relu', kernel_initializer = 'he_uniform'))
model.add(Dense(10, activation='softmax'))

#defining the batch sizes and epochs
batch_size = 64
epochs = 60
opt = SGD()

In [31]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
early_stop= EarlyStopping(monitor='val_loss',patience=2)

In [33]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_66 (Conv2D)           (None, 67, 67, 32)        896       
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 67, 67, 32)        9248      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 33, 33, 32)        0         
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 33, 33, 64)        18496     
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 33, 33, 64)        36928     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 16, 16, 128)      

In [34]:
#training the model
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

Epoch 1/60
307/307 [==============================] - 11s 32ms/step - loss: 1.7717 - accuracy: 0.3303 - val_loss: 2.4651 - val_accuracy: 0.3034
Epoch 2/60
307/307 [==============================] - 10s 32ms/step - loss: 1.4782 - accuracy: 0.4386 - val_loss: 1.1277 - val_accuracy: 0.5833
Epoch 3/60
307/307 [==============================] - 10s 31ms/step - loss: 1.1695 - accuracy: 0.5680 - val_loss: 1.1663 - val_accuracy: 0.5879
Epoch 4/60
307/307 [==============================] - 10s 31ms/step - loss: 1.0036 - accuracy: 0.6301 - val_loss: 1.3544 - val_accuracy: 0.4608
Epoch 5/60
307/307 [==============================] - 10s 31ms/step - loss: 0.9228 - accuracy: 0.6537 - val_loss: 1.0695 - val_accuracy: 0.6370
Epoch 6/60
307/307 [==============================] - 9s 31ms/step - loss: 0.8373 - accuracy: 0.6909 - val_loss: 0.9069 - val_accuracy: 0.6673
Epoch 7/60
307/307 [==============================] - 9s 31ms/step - loss: 0.8221 - accuracy: 0.6950 - val_loss: 0.8231 - val_accuracy: 0

In [35]:
#saving the model
model.save("astroNN2.h5")

In [36]:
#evaluating model over test dataset
model.evaluate(X_test, y_test)

69/69 [==============================] - 1s 7ms/step - loss: 1.7637 - accuracy: 0.7760


[1.7636834383010864, 0.7760440707206726]

#Prediction

In [37]:
#displaying the predictions along with the actual values
for i in range(10):

    img = images[i]

    img = np.expand_dims(img,0) # <--- add batch axis
    #print(img.shape)
    output = model.predict(img)
    #print(output[0])
    predictedClass = np.argmax(output[0])

    print("\nPredicted: ",galaxy10cls_lookup(predictedClass))
    print("Actual: ",galaxy10cls_lookup(np.argmax(labels[i])))
    print("-------------------------------------------------------------------------------------------------------------------------------------------")


Predicted:  Smooth, in-between round
Actual:  Smooth, in-between round
-------------------------------------------------------------------------------------------------------------------------------------------

Predicted:  Smooth, in-between round
Actual:  Smooth, in-between round
-------------------------------------------------------------------------------------------------------------------------------------------

Predicted:  Disk, Edge-on, Rounded Bulge
Actual:  Disk, Edge-on, Rounded Bulge
-------------------------------------------------------------------------------------------------------------------------------------------

Predicted:  Smooth, in-between round
Actual:  Smooth, in-between round
-------------------------------------------------------------------------------------------------------------------------------------------

Predicted:  Disk, Edge-on, Rounded Bulge
Actual:  Disk, Edge-on, Rounded Bulge
----------------------------------------------------------------

#Classification Report and Confusion Matrix

In [38]:
#displaying classification report
y_pred=model.predict_classes(X_test)
y_pred

#converting the one-hot encoded labels back
rounded_labels=np.argmax(y_test, axis=1)

print('Classification Report')
print(classification_report(rounded_labels,y_pred))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Classification Report
              precision    recall  f1-score   support

           0       0.56      0.52      0.54       360
           1       0.90      0.93      0.91       703
           2       0.84      0.89      0.86       599
           3       0.47      0.40      0.43        35
           4       0.83      0.87      0.85       151
           5       0.00      0.00      0.00         3
           6       0.77      0.77      0.77        64
           7       0.51      0.44      0.48       126
           8       0.54      0.42      0.47        90
           9       0.62      0.60      0.61        48

    accuracy                           0.78      2179
   macro avg       0.60      0.58      0.59      2179
weighted avg       0.76      0.78      0.77      2179



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
#displaying confusion matrix
print('Confusion Matrix')
print(confusion_matrix(rounded_labels,y_pred))

Confusion Matrix
[[188  47  60   5   4   0   1  28  16  11]
 [ 23 654  21   0   0   0   0   5   0   0]
 [ 38  20 532   0   1   0   0   5   2   1]
 [  5   0   3  14   4   0   8   0   0   1]
 [  3   1   5   4 131   0   6   0   0   1]
 [  0   0   0   0   3   0   0   0   0   0]
 [  0   0   0   4  11   0  49   0   0   0]
 [ 35   5  14   2   1   0   0  56  12   1]
 [ 32   0   2   0   0   0   0  15  38   3]
 [ 11   2   0   1   2   0   0   0   3  29]]
